In [1]:
import pandas as pd
import numpy as np

### Сырой DataFrame

In [2]:
df_raw = pd.read_csv("../../resources/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
df_raw.columns=["unixtimestamp", "open", "high","low","close", "volume_btc","volume_usd","weighted_price"]

### Чистка данных

In [3]:
df = df_raw.dropna()
df = df.loc[(df["volume_btc"] > 0) & (df["high"] != df["low"])]
df.insert(loc=0, column='timestamp', value=pd.to_datetime(df['unixtimestamp'],unit = 's'))
df.reset_index(inplace=True, drop=True)

In [4]:
df.head()

,timestamp,unixtimestamp,open,high,low,close,volume_btc,volume_usd,weighted_price
0,2011-12-31 16:59:00,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
1,2012-01-04 16:00:00,1325692800,5.36,5.37,5.36,5.37,13.629423,73.060000,5.360462
2,2012-01-04 17:51:00,1325699460,5.37,5.57,5.37,5.57,43.312196,235.747069,5.442972
3,2012-01-05 07:19:00,1325747940,5.75,5.79,5.75,5.79,14.800000,85.500000,5.777027
4,2012-01-05 10:10:00,1325758200,6.19,6.23,6.19,6.23,16.000000,99.285719,6.205357


### Расширение данных

#### SMA

In [5]:
df["sma14"] = df["close"].rolling(14).mean()
df["sma30"] = df["close"].rolling(30).mean()

#### EMA

In [6]:
# smoothing=2
# period12=12
# period26=26
# c_ema12=smoothing/(period12 + 1)
# c_ema26=smoothing/(period26 + 1)
# ema12 = df["close"] * c_ema12
# ema12.iloc[:period12] = np.NaN
# ema26 = df["close"] * c_ema26
# ema26.iloc[:period26] = np.NaN

In [7]:
# ema12.iloc[period12] = df["close"].iloc[period12] * c_ema12 + df["close"].iloc[:period12].mean() * (1 - c_ema12)
# ema26.iloc[period26] = df["close"].iloc[period26] * c_ema26 + df["close"].iloc[:period26].mean() * (1 - c_ema26)

In [8]:
# for i in df.index[period12+1:period26+1]:
#     ema12.iloc[i] = df["close"].iloc[i] * c_ema12 + ema12.iloc[i-1] * (1 - c_ema12)
# for i in df.index[period26+1:]:
#     close = df["close"].iloc[i]
#     ema12.iloc[i] = close * c_ema12 + ema12.iloc[i-1] * (1 - c_ema12)
#     ema26.iloc[i] = close * c_ema26 + ema26.iloc[i-1] * (1 - c_ema26)

In [9]:
df["ema12"] = df["close"].ewm(span=12, adjust=False).mean()
df["ema26"] = df["close"].ewm(span=26, adjust=False).mean()

#### MACD

In [10]:
df["macd"] = df["ema12"] - df["ema26"]

#### OBV

In [11]:
def obv(obv, vol, close, close_prev1):
    if close > close_prev1:
        return obv + vol
    elif close < close_prev1:
        return obv - vol
    else:
        return obv

def obv_acc(df):
    def obv_value(x):
        if x["close"] > x["close_prev1"]:
            return x["volume_btc"]
        elif x["close"] < x["close_prev1"]:
            return -x["volume_btc"]
        else:
            return 0
    return df.apply(obv_value, axis=1).cumsum()

In [12]:
df_obv_input = df[["volume_btc", "close"]]
df_obv_input["close_prev1"] = df_obv_input["close"].shift(1)
df["obv"] = obv_acc(df_obv_input)

<ipython-input-12-c4bca509a06d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_obv_input["close_prev1"] = df_obv_input["close"].shift(1)


#### Accumulation/Distirbution
Индикатор можно использовать для подтверждения тренда. Если растет цена и увеличивается индикатор, то тренд нарастающий

In [13]:
def ad_acc(df):
    mfm = df.apply(lambda x: ((x["close"] - x["low"]) - (x["high"] - x["close"]))/(x["high"] - x["low"]), axis=1)
    mfv = df["volume_btc"] * mfm
    return mfv.cumsum()

In [14]:
ad_raw = ad_acc(df)
df["ad"] = ad_raw

#### True Range + Average True Range

In [15]:
tr = pd.DataFrame({
    "0": df["high"] - df["low"],
    "1": (df["high"] - df["close"]).abs(),
    "2": (df["low"] - df["close"]).abs(),
}).max(axis=1)
atr = tr.rolling(14, min_periods=1).mean()

#### Average Directional Index (ADX)

In [16]:
df_prev1 = df.shift(1)
pos_dm = df["high"] - df_prev1["high"]
sm_pos_dm = pos_dm.rolling(14, min_periods=1).sum() - pos_dm.rolling(14, min_periods=1).mean() + pos_dm
pos_di = sm_pos_dm / atr * 100
neg_dm = df["low"] - df_prev1["low"]
sm_neg_dm = neg_dm.rolling(14, min_periods=1).sum() - neg_dm.rolling(14, min_periods=1).mean() + neg_dm
neg_di = sm_neg_dm / atr * 100

In [17]:
dx = 100 * (pos_di - neg_di).abs() / (pos_di + neg_di).abs()

In [18]:
adx_easy = dx.rolling(14).mean()
df["adx"] = adx_easy

Не уверен, что нужно вот так заморачиваться. Считать по новым данным - да, для исторических - хз

In [19]:
# adx = dx.copy()
# adx.loc[:14] = np.NaN
# adx.loc[15] = dx[:14].mean()
# adx.loc[15:] = list(map(lambda i: (adx[i-1] * 13 + adx[i]) / 14, adx.index[14:]))
# df["adx"] = adx

#### Aaron oscillator

In [20]:
period_since_max = df["high"].rolling(25, min_periods=1).apply(lambda x: x.argmax(), raw=True)
aaron_up = (25 - period_since_max)/25 * 100
period_since_min = df["low"].rolling(25, min_periods=1).apply(lambda x: x.argmin(), raw=True)
aaron_down = (25 - period_since_min)/25 * 100
aaron_osc = aaron_up - aaron_down
df["aaron"] = aaron_osc

#### Relative Strength Index

In [21]:
# avg_gain = df.iloc[:14].loc[df["close"] > df["open"]]
# avg_gain = ((avg_gain["close"] - avg_gain["open"])/avg_gain["open"] * 100).mean()

# avg_loss = df.iloc[:14].loc[df["close"] < df["open"]]
# avg_loss = ((avg_loss["open"] - avg_loss["close"])/avg_loss["open"] * 100).mean()

# rsi_0 = 100 - 100/(1 + avg_gain/avg_loss)

In [22]:
# rsi_df = df[["open", "close"]]
# rsi_df.loc[14, "rsi"] = rsi_0
# rsi_df.loc[14, "avg_loss"] = avg_loss
# rsi_df.loc[14, "avg_gain"] = avg_gain

In [23]:
gains, losses = (df["close"] - df["open"])/df["open"]*100, (df["open"] - df["close"])/df["open"]*100
gains[gains < 0] = 0
losses[losses < 0] = 0
df["rsi"] = 100 - 100/(1 + gains.rolling(14).mean() / losses.rolling(14).mean())
df["rsi"]

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
             ...    
2860480    47.204688
2860481    44.492785
2860482    51.085974
2860483    54.282463
2860484    55.931482
Name: rsi, Length: 2860485, dtype: float64

#### Stochastic Oscillator

In [24]:
df["so"] = 100*(df["close"].shift(1) - df["low"].rolling(14).min())/(df["high"].rolling(14).max() - df["low"].rolling(14).min())
df["so"].tail(20)

2860465    28.305603
2860466    35.681335
2860467    22.194578
2860468     8.545830
2860469     5.156494
2860470     0.131910
2860471     0.131910
2860472    40.028780
2860473    61.764000
2860474    49.897541
2860475    33.943058
2860476    69.525434
2860477    76.165843
2860478    60.694743
2860479    86.178912
2860480    70.415002
2860481    53.009530
2860482    52.892714
2860483    72.686191
2860484    87.988255
Name: so, dtype: float64

### Учтем вчерашние значения + относительное изменения в %

In [25]:
columns_to_shift = ['open', 'high', 'low', 'close',
       'volume_btc', 'volume_usd', 'weighted_price', 'sma14', 'sma30', 'ema12',
       'ema26', 'macd', 'obv', 'ad', 'adx', 'aaron', 'rsi', 'so']
df_shifted = df[columns_to_shift].shift(1)
df[list(map(lambda col: col + "_prev1", columns_to_shift))] = df_shifted
df[list(map(lambda col: col + "_rel", columns_to_shift))] = df[columns_to_shift] / df_shifted

### Сдвинем относительные изменения из предыдущего пункта

In [26]:
rel_columns_to_shift = list(map(lambda col: col + "_rel", columns_to_shift))
df_shifted = df[rel_columns_to_shift].shift(1)
df[list(map(lambda col: col + "_prev1", rel_columns_to_shift))] = df_shifted

### Готово

In [27]:
df.dropna()
df.to_hdf("btc_data.hdf", "table")